# Masterlist overview

This notebook summarises some important properties of the data presented here. This includes

- The area covered by each field
- The number of total sources in each field
- The number of bands which we have data for in each field
- The size of the masterlist in each field

In [1]:
from astropy.table import Table, Column
from astropy import units as u
import numpy as np

from pymoc import MOC

from herschelhelp_internal.masterlist import find_last_ml_suffix

import os
import humanfriendly

/Users/rs548/anaconda/envs/herschelhelp_internal/lib/python3.6/site-packages/seaborn/apionly.py:6: UserWarning: As seaborn no longer sets a default style on import, the seaborn.apionly module is deprecated. It will be removed in a future version.
  warnings.warn(msg, UserWarning)


In [2]:
humanfriendly.format_size(524255040.0)

'524.26 MB'

In [3]:
dr1 = [
# Field                suffix
['AKARI-NEP',          '20171202' ],
['AKARI-SEP',          '20180122' ],
['Bootes',             '20180111' ],
['CDFS-SWIRE',         '20180129' ],
['COSMOS',             '20180112' ],
['EGS',                '20171202' ],
['ELAIS-N1',           '20171016' ],
['ELAIS-N2',           '20171207' ],
['ELAIS-S1',           '20180122' ],
['GAMA-09',            '20171206' ],
['GAMA-12',            '20180129' ],
['GAMA-15',            '20180129' ],
['HDF-N',              '20171212' ],
['Herschel-Stripe-82', '20180113' ],
['Lockman-SWIRE',      '20171201' ],
['NGP',                '20171207' ],
['SA13',               '20171123' ],
['SGP',                '20180124' ],
['SPIRE-NEP',          '20171122' ],
['SSDF',               '20180122' ],
['xFLS',               '20171201' ],
['XMM-13hr',           '20171123' ],
['XMM-LSS',            '20180130' ]]

In [7]:
masterlist_summary = Table()
masterlist_summary.add_column(Column(data = np.array(dr1)[:,0], name = 'field'))
masterlist_summary.add_column(Column(data =np.full(len(masterlist_summary), '', dtype="S20"), name = 'dr1_suffix'))
masterlist_summary.add_column(Column(data =np.full(len(masterlist_summary), '', dtype="S20"), name = 'last_suffix'))
masterlist_summary.add_column(Column(data =np.full(len(masterlist_summary), 0, dtype=int), name = 'objects'))
masterlist_summary.add_column(Column(data =np.full(len(masterlist_summary), np.nan, dtype=float), name = 'area_sq_degrees'))
masterlist_summary.add_column(Column(data =np.full(len(masterlist_summary), 0, dtype=int), name = 'file_size_bytes'))
masterlist_summary.add_column(Column(data =np.full(len(masterlist_summary), '', dtype="S20"), name = 'file_size_readable'))

In [5]:
help_moc = MOC(filename='../dmu2/help_coverage_MOC.fits')
print('Total HELP area: {} sqare degrees'.format(round(help_moc.area_sq_deg)))

Total HELP area: 1270 sqare degrees


In [6]:
for field in dr1:
    number_objects = len(Table.read('dmu1_ml_{}/data/master_list_cross_ident_{}_{}.fits'.format(field[0], 
                                                                                      field[0].lower(), 
                                                                                      field[1])))
    masterlist_summary['objects'][masterlist_summary['field'] == field[0]] = number_objects
    
    
    moc = MOC(filename='../dmu2/dmu2_field_coverages/{}_MOC.fits'.format(field[0]))
    area = round(moc.area_sq_deg)
    masterlist_summary['area_sq_degrees'][masterlist_summary['field'] == field[0]] = area
    
    masterlist_summary['dr1_suffix'][masterlist_summary['field'] == field[0]] = field[1]
    masterlist_summary['last_suffix'][masterlist_summary['field'] == field[0]] = find_last_ml_suffix('dmu1_ml_{}/data/'.format(field[0]))
    
    size = os.stat('dmu1_ml_{}/data/master_catalogue_{}_{}.fits'.format(field[0], 
                                                                                      field[0].lower(), 
                                                                                      field[1])).st_size

    
    
    #Can't install packages on Vulcain
    #size_readable = humanfriendly.format_size(size)
    masterlist_summary['file_size_bytes'][masterlist_summary['field'] == field[0]] = size
    masterlist_summary['file_size_readable'][masterlist_summary['field'] == field[0]] = humanfriendly.format_size(size)
    

    
    #TODO:
    num_surveys = 0.
    
    #Mean depth in g
    mean_depth = 0.
    
    num_bands = 0.

In [7]:
masterlist_summary.show_in_notebook()

idx,field,dr1_suffix,last_suffix,objects,area_sq_degrees,file_size_bytes,file_size_readable
0,AKARI-NEP,2,2,285786,9.0,241231680,2
1,AKARI-SEP,2,2,844172,9.0,524255040,5
2,Bootes,2,2,3388482,11.0,4537221120,4
3,CDFS-SWIRE,2,2,2171051,13.0,5731646400,5
4,COSMOS,2,2,2779194,5.0,4630199040,4
5,EGS,2,2,1412613,4.0,3890416320,3
6,ELAIS-N1,2,2,4026292,14.0,5242279680,5
7,ELAIS-N2,2,2,1783240,9.0,1941984000,1
8,ELAIS-S1,2,2,1655564,9.0,1908901440,1
9,GAMA-09,2,2,12937982,62.0,19652852160,1


In [8]:
print('Total number of objects in HELP: {}'.format(np.sum(masterlist_summary['objects'])))

Total number of objects in HELP: 170619898


In [9]:
print('Total size of masterlist files: {}'.format(humanfriendly.format_size(np.sum(masterlist_summary['file_size_bytes']))))

Total size of masterlist files: 291.82 GB


In [10]:
masterlist_summary.write('dr1_overview.fits', overwrite=True)